In [1]:
import argparse
import os
import time
import shutil

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn


import torchvision
import torchvision.transforms as transforms

from models import *


global best_prec
use_gpu = torch.cuda.is_available()
print('=> Building model...')



batch_size = 128
model_name = "ResNet20_project"
model = resnet20_internal_8x8()

print(model)

normalize = transforms.Normalize(mean=[0.491, 0.482, 0.447], std=[0.247, 0.243, 0.262])


train_dataset = torchvision.datasets.CIFAR10(
    root='./data',
    train=True,
    download=True,
    transform=transforms.Compose([
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        normalize,
    ]))
trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)


test_dataset = torchvision.datasets.CIFAR10(
    root='./data',
    train=False,
    download=True,
    transform=transforms.Compose([
        transforms.ToTensor(),
        normalize,
    ]))

testloader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)


print_freq = 100 # every 100 batches, accuracy printed. Here, each batch includes "batch_size" data points
# CIFAR10 has 50,000 training data, and 10,000 validation data.

def train(trainloader, model, criterion, optimizer, epoch):
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()

    model.train()

    end = time.time()
    for i, (input, target) in enumerate(trainloader):
        # measure data loading time
        data_time.update(time.time() - end)

        input, target = input.cuda(), target.cuda()

        # compute output
        output = model(input)
        loss = criterion(output, target)

        # measure accuracy and record loss
        prec = accuracy(output, target)[0]
        losses.update(loss.item(), input.size(0))
        top1.update(prec.item(), input.size(0))

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()


        if i % print_freq == 0:
            print('Epoch: [{0}][{1}/{2}]\t'
                  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Data {data_time.val:.3f} ({data_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                  'Prec {top1.val:.3f}% ({top1.avg:.3f}%)'.format(
                   epoch, i, len(trainloader), batch_time=batch_time,
                   data_time=data_time, loss=losses, top1=top1))



def validate(val_loader, model, criterion ):
    batch_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()

    # switch to evaluate mode
    model.eval()

    end = time.time()
    with torch.no_grad():
        for i, (input, target) in enumerate(val_loader):

            input, target = input.cuda(), target.cuda()

            # compute output
            output = model(input)
            loss = criterion(output, target)

            # measure accuracy and record loss
            prec = accuracy(output, target)[0]
            losses.update(loss.item(), input.size(0))
            top1.update(prec.item(), input.size(0))

            # measure elapsed time
            batch_time.update(time.time() - end)
            end = time.time()

            if i % print_freq == 0:  # This line shows how frequently print out the status. e.g., i%5 => every 5 batch, prints out
                print('Test: [{0}/{1}]\t'
                  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                  'Prec {top1.val:.3f}% ({top1.avg:.3f}%)'.format(
                   i, len(val_loader), batch_time=batch_time, loss=losses,
                   top1=top1))

    print(' * Prec {top1.avg:.3f}% '.format(top1=top1))
    return top1.avg


def accuracy(output, target, topk=(1,)):
    """Computes the precision@k for the specified values of k"""
    maxk = max(topk)
    batch_size = target.size(0)

    _, pred = output.topk(maxk, 1, True, True)
    pred = pred.t()
    correct = pred.eq(target.view(1, -1).expand_as(pred))

    res = []
    for k in topk:
        correct_k = correct[:k].view(-1).float().sum(0)
        res.append(correct_k.mul_(100.0 / batch_size))
    return res


class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count


def save_checkpoint(state, is_best, fdir):
    filepath = os.path.join(fdir, 'checkpoint.pth')
    torch.save(state, filepath)
    if is_best:
        shutil.copyfile(filepath, os.path.join(fdir, 'model_best.pth.tar'))

#model = nn.DataParallel(model).cuda()
#all_params = checkpoint['state_dict']
#model.load_state_dict(all_params, strict=False)
#criterion = nn.CrossEntropyLoss().cuda()
#validate(testloader, model, criterion)

=> Building model...
ResNet_Cifar_Internal_8x8(
  (conv1): QuantConv2d(
    3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
    (weight_quant): weight_quantize_fn()
  )
  (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): QuantConv2d(
        16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
        (weight_quant): weight_quantize_fn()
      )
      (conv2): QuantConv2d(
        16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
        (weight_quant): weight_quantize_fn()
      )
      (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): QuantConv2d(
        16, 16, kernel_size=(3, 3), stride=(1, 1), padd

In [2]:
def adjust_learning_rate(optimizer, epoch):
    """For resnet, the lr starts from 0.1, and is divided by 10 at 80 and 120 epochs"""
    adjust_list = [60, 80]
    if epoch in adjust_list:
        for param_group in optimizer.param_groups:
            param_group['lr'] = param_group['lr'] * 0.1

In [3]:
# This cell won't be given, but students will complete the training

lr = 4e-2
weight_decay = 1e-4
epochs = 100
best_prec = 0

#model = nn.DataParallel(model).cuda()
model.cuda()
criterion = nn.CrossEntropyLoss().cuda()
optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=0.9, weight_decay=weight_decay)
#cudnn.benchmark = True

if not os.path.exists('result'):
    os.makedirs('result')
fdir = 'result/'+str(model_name)
if not os.path.exists(fdir):
    os.makedirs(fdir)


for epoch in range(0, epochs):
    adjust_learning_rate(optimizer, epoch)

    train(trainloader, model, criterion, optimizer, epoch)

    # evaluate on test set
    print("Validation starts")
    prec = validate(testloader, model, criterion)

    # remember best precision and save checkpoint
    is_best = prec > best_prec
    best_prec = max(prec,best_prec)
    print('best acc: {:1f}'.format(best_prec))
    save_checkpoint({
        'epoch': epoch + 1,
        'state_dict': model.state_dict(),
        'best_prec': best_prec,
        'optimizer': optimizer.state_dict(),
    }, is_best, fdir)

Epoch: [0][0/391]	Time 1.909 (1.909)	Data 0.143 (0.143)	Loss 2.3821 (2.3821)	Prec 6.250% (6.250%)
Epoch: [0][100/391]	Time 0.032 (0.052)	Data 0.001 (0.003)	Loss 1.8416 (1.9802)	Prec 36.719% (24.435%)
Epoch: [0][200/391]	Time 0.034 (0.042)	Data 0.002 (0.002)	Loss 1.6746 (1.8451)	Prec 35.938% (29.878%)
Epoch: [0][300/391]	Time 0.033 (0.039)	Data 0.001 (0.002)	Loss 1.6057 (1.7557)	Prec 38.281% (33.591%)
Validation starts
Test: [0/79]	Time 0.112 (0.112)	Loss 1.4880 (1.4880)	Prec 46.875% (46.875%)
 * Prec 45.050% 
best acc: 45.050000
Epoch: [1][0/391]	Time 0.146 (0.146)	Data 0.104 (0.104)	Loss 1.4578 (1.4578)	Prec 50.781% (50.781%)
Epoch: [1][100/391]	Time 0.033 (0.036)	Data 0.002 (0.003)	Loss 1.3928 (1.3908)	Prec 52.344% (49.002%)
Epoch: [1][200/391]	Time 0.034 (0.035)	Data 0.002 (0.002)	Loss 1.4572 (1.3708)	Prec 47.656% (49.580%)
Epoch: [1][300/391]	Time 0.032 (0.034)	Data 0.001 (0.002)	Loss 1.1810 (1.3377)	Prec 57.812% (51.041%)
Validation starts
Test: [0/79]	Time 0.092 (0.092)	Loss 1.23

In [4]:
PATH = "/content/result/ResNet20_project/model_best.pth.tar"
model_name = "ResNet20_project"
model = resnet20_internal_8x8()
checkpoint = torch.load(PATH)
model.load_state_dict(checkpoint['state_dict'])
device = torch.device("cuda")

model.cuda()
model.eval()

test_loss = 0
correct = 0

with torch.no_grad():
    for data, target in testloader:
        data, target = data.to(device), target.to(device) # loading to GPU
        output = model(data)
        pred = output.argmax(dim=1, keepdim=True)
        correct += pred.eq(target.view_as(pred)).sum().item()

test_loss /= len(testloader.dataset)

print('\nTest set: Accuracy: {}/{} ({:.0f}%)\n'.format(
        correct, len(testloader.dataset),
        100. * correct / len(testloader.dataset)))


Test set: Accuracy: 8958/10000 (90%)



In [10]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class SaveOutput:
    def __init__(self):
        self.outputs = []

    def __call__(self, module, module_in):
        self.outputs.append(module_in[0])

    def clear(self):
        self.outputs = []

save_output = SaveOutput()
hook_handles = []

print("Registering hooks...")
for i, m in enumerate(model.modules()):
    if isinstance(m, QuantConv2d):
        handle = m.register_forward_pre_hook(save_output)
        hook_handles.append(handle)


model.eval()
model.cuda()


dataiter = iter(testloader)
images, labels = next(dataiter)
images = images.cuda()

save_output.clear()

with torch.no_grad():
    _ = model(images)

print(f"Captured {len(save_output.outputs)} layer inputs.")


input_idx = 10
ref_idx = 11

x_raw = save_output.outputs[input_idx].detach()


target_layer = model.layer2[1].conv1
weight_q = target_layer.weight_q.detach()


output_ref = save_output.outputs[ref_idx].detach()

print(f"\n--- Data Shapes ---")
print(f"Input:  {x_raw.shape}")
print(f"Weight: {weight_q.shape}")
print(f"Ref:    {output_ref.shape}")


x_bit = 4
w_bit = 4

x_alpha = target_layer.act_alpha.detach()
x_delta = x_alpha / (2**x_bit - 1)

act_quant_fn = act_quantization(x_bit)
x_quantized_float = act_quant_fn(x_raw, x_alpha)
x_int = x_quantized_float / x_delta


w_alpha = target_layer.weight_quant.wgt_alpha.detach()
w_delta = w_alpha / (2**(w_bit - 1) - 1)
w_int = weight_q / w_delta

print(f"\n--- Integer Check ---")
print(f"x_int sample: {x_int[0,0,0,:5].tolist()}")
print(f"w_int sample: {w_int[0,0,0,:5].tolist()}")


sim_conv = nn.Conv2d(in_channels=8, out_channels=8, kernel_size=3, padding=1, bias=False).cuda()
sim_conv.weight = nn.Parameter(w_int)

output_int = sim_conv(x_int)

output_recovered = output_int * x_delta * w_delta
output_simulated = F.relu(output_recovered)

difference = (output_simulated - output_ref).abs()
mean_error = difference.mean().item()

print(f"\n--- Verification Result ---")
print(f"Mean Difference: {mean_error}")



for h in hook_handles:
    h.remove()
save_output.clear()

Registering hooks...
Captured 22 layer inputs.

--- Data Shapes ---
Input:  torch.Size([128, 8, 16, 16])
Weight: torch.Size([8, 8, 3, 3])
Ref:    torch.Size([128, 8, 16, 16])

--- Integer Check ---
x_int sample: [9.0, 4.0, 4.0, 3.000000238418579, 3.000000238418579]
w_int sample: [4.999999523162842, 2.0, 0.0]

--- Verification Result ---
Mean Difference: 1.0696428489609389e-06


In [ ]:
x_pad = torch.zeros(128, 8, 6, 6).to(x_int.device)
x_pad[:, :, 1:5, 1:5] = x_int
X = x_pad[0]
X = X.reshape(X.size(0), -1)
X.size()

torch.Size([8, 36])

In [ ]:
tile_id = 0
nij = 200 # just a random number
#X = a_tile[tile_id,:,nij:nij+64]  # [tile_num, array row num, time_steps]

bit_precision = 4
file = open('activation.txt', 'w') #write to file
file.write('#time0row7[msb-lsb],time0row6[msb-lst],....,time0row0[msb-lst]#\n')
file.write('#time1row7[msb-lsb],time1row6[msb-lst],....,time1row0[msb-lst]#\n')
file.write('#................#\n')

for i in range(X.size(1)):  # time step
    for j in range(X.size(0)): # row #
        X_bin = '{0:04b}'.format(int(X[7-j,i].item()+0.001))
        for k in range(bit_precision):
            file.write(X_bin[k])
        #file.write(' ')  # for visibility with blank between words, you can use
    file.write('\n')
file.close() #close file

In [ ]:
print(weight_int.size())
W = torch.reshape(weight_int, (weight_int.size(0), weight_int.size(1), -1))
print(W.size())

torch.Size([8, 8, 3, 3])
torch.Size([8, 8, 9])


In [ ]:

bit_precision = 4
file = open('weight.txt', 'w') #write to file
file.write('#col0row7[msb-lsb],col0row6[msb-lst],....,col0row0[msb-lst]#\n')
file.write('#col1row7[msb-lsb],col1row6[msb-lst],....,col1row0[msb-lst]#\n')
file.write('#................#\n')
for kij in range(9):
    for i in range(W.size(0)):
        for j in range(W.size(1)):
            if (W[i, 7-j, kij].item()<0):
                W_bin = '{0:04b}'.format(int(W[i,7-j,kij].item()+2**bit_precision+0.001))
            else:
                W_bin = '{0:04b}'.format(int(W[i,7-j,kij].item()+0.001))
            for k in range(bit_precision):
                file.write(W_bin[k])
            #file.write(' ')  # for visibility with blank between words, you can use
        file.write('\n')
file.close() #close file

In [ ]:
W[0,:,0]

tensor([-5.0000, -0.0000,  2.0000,  7.0000,  7.0000,  3.0000, -3.0000,  2.0000],
       device='cuda:0', grad_fn=<SelectBackward0>)

In [ ]:
p_nijg = range(X.size(1)) ## psum nij group
psum = torch.zeros(8, len(p_nijg), 9).cuda()
for kij in range(9):
    for nij in p_nijg:       # time domain, sequentially given input
        m = nn.Linear(8, 8, bias=False)
        m.weight = torch.nn.Parameter(W[:,:,kij])
        psum[:, nij, kij] = m(X[:,nij]).cuda()
bit_precision = 16
file = open('psum.txt', 'w') #write to file
file.write('#time0col7[msb-lsb],time0col6[msb-lst],....,time0col0[msb-lst]#\n')
file.write('#time1col7[msb-lsb],time1col6[msb-lst],....,time1col0[msb-lst]#\n')
file.write('#................#\n')
for kij in range(9):
    for i in range(psum.size(1)):
        for j in range(psum.size(0)):
            if (psum[7-j,i,kij].item()<0):
                P_bin = '{0:016b}'.format(int(psum[7-j,i,kij].item()+2**bit_precision+0.001))
            else:
                P_bin = '{0:016b}'.format(int(psum[7-j,i,kij].item()+0.001))
            for k in range(bit_precision):
                file.write(P_bin[k])
            #file.write(' ')  # for visibility with blank between words, you can use
        file.write('\n')
file.close()

In [ ]:
out = relu(output_int[0])
out = torch.reshape(out, (out.size(0), -1))
bit_precision = 16
file = open('output.txt', 'w') #write to file
file.write('#time0col7[msb-lsb],time0col6[msb-lst],....,time0col0[msb-lst]#\n')
file.write('#time1col7[msb-lsb],time1col6[msb-lst],....,time1col0[msb-lst]#\n')
file.write('#................#\n')

for i in range(out.size(1)):
    for j in range(out.size(0)):
        if (out[7-j,i].item()<0):
            O_bin = '{0:016b}'.format(int(out[7-j,i].item()+2**bit_precision+0.001))
        else:
            O_bin = '{0:016b}'.format(int(out[7-j,i].item()+0.001))
        for k in range(bit_precision):
            file.write(O_bin[k])
        #file.write(' ')  # for visibility with blank between words, you can use
    file.write('\n')
file.close()

In [ ]:
import torch
import math

# ======================
# 1. 从 activation.txt 读回 x_pad[0]
# ======================

def read_activation_txt(path, C_in=8):
    lines = []
    with open(path, 'r') as f:
        for line in f:
            line = line.strip()
            if not line or line.startswith('#'):
                continue
            lines.append(line)

    T = len(lines)                 # time steps = 展平后的空间点数
    X = torch.zeros(C_in, T, dtype=torch.int32)

    for t, line in enumerate(lines):
        # 每行: ch7(4bit) ch6(4bit) ... ch0(4bit) → 共 8*4 = 32 bit
        assert len(line) >= C_in * 4, f"activation line too short: {len(line)} bits"

        for c in range(C_in):
            group_idx = 7 - c      # 写入时用了 X[7-j,i]，所以 group=7-c
            start = group_idx * 4
            bits = line[start:start+4]
            val = int(bits, 2)     # 无符号 4bit（ReLU 后不会是负的）
            X[c, t] = val

    # 还原空间尺寸: T = H_pad * W_pad
    H_pad = int(math.sqrt(T))
    W_pad = H_pad
    assert H_pad * W_pad == T, f"activation T={T} 不能 sqrt 成方形"

    x_pad_0 = X.view(C_in, H_pad, W_pad)  # [C_in, H_pad, W_pad]
    return x_pad_0


# ======================
# 2. 从 weight.txt 读回 weight_int
# ======================

def read_weight_txt(path, C_in=8, K=3):
    lines = []
    with open(path, 'r') as f:
        for line in f:
            line = line.strip()
            if not line or line.startswith('#'):
                continue
            lines.append(line)

    total_lines = len(lines)
    K2 = K * K                      # 3x3 → 9
    assert total_lines % K2 == 0, "weight.txt 行数不是 9 的整数倍，检查文件格式"

    C_out = total_lines // K2       # 每个 kij 有 C_out 行
    W = torch.zeros(C_out, C_in, K2, dtype=torch.int32)  # [oc, ic, kij]

    for kij in range(K2):
        for oc in range(C_out):
            line_idx = kij * C_out + oc
            line = lines[line_idx]
            assert len(line) >= C_in * 4, f"weight line too short: {len(line)} bits"

            for ic in range(C_in):
                group_idx = 7 - ic       # 写入时 W[i,7-j,kij]，group = j → ic = 7-j
                start = group_idx * 4
                bits = line[start:start+4]
                v_u = int(bits, 2)       # 0..15

                # 4-bit 二补码 → 有符号
                if v_u >= 8:
                    v = v_u - 16
                else:
                    v = v_u

                W[oc, ic, kij] = v

    # 还原到 [C_out, C_in, K, K]
    weight_int = W.view(C_out, C_in, K, K)
    return weight_int


# ======================
# 3. 从 output.txt 读回 output_int[0]
# ======================

def read_output_txt(path, C_out=8):
    lines = []
    with open(path, 'r') as f:
        for line in f:
            line = line.strip()
            if not line or line.startswith('#'):
                continue
            lines.append(line)

    T = len(lines)                  # 输出的空间点数 (4x4=16)
    out_flat = torch.zeros(C_out, T, dtype=torch.int32)

    for t, line in enumerate(lines):
        assert len(line) >= C_out * 16, f"output line too short: {len(line)} bits"
        for oc in range(C_out):
            group_idx = 7 - oc      # 写入时 out[7-j,i]，同理 group=7-oc
            start = group_idx * 16
            bits = line[start:start+16]
            v_u = int(bits, 2)      # 无符号 16-bit

            # 16-bit 二补码
            if v_u >= 2**15:
                v = v_u - 2**16
            else:
                v = v_u

            out_flat[oc, t] = v

    # 还原空间 H_out x W_out：这里我们知道是 4x4，如果不确定可以 sqrt
    H_out = int(math.sqrt(T))
    W_out = H_out
    assert H_out * W_out == T, f"output T={T} 不能 sqrt 成方形"

    out_0 = out_flat.view(C_out, H_out, W_out)  # [C_out, H_out, W_out]
    return out_0


# ======================
# 4. 用重建的 x_pad_0 和 weight_int 手动做 3x3 conv
# ======================

def manual_conv_3x3_stride1_pad1_from_xpad(x_pad_0, weight_int):
    """
    x_pad_0: [C_in, H_pad, W_pad] (比如 8x6x6)
    weight_int: [C_out, C_in, 3, 3]
    return: [C_out, H_out, W_out] = [C_out, H_pad-2, W_pad-2]
    """
    C_in = x_pad_0.size(0)
    C_out = weight_int.size(0)
    Kh = Kw = 3
    H_pad, W_pad = x_pad_0.size(1), x_pad_0.size(2)
    H_out, W_out = H_pad - Kh + 1, W_pad - Kw + 1

    out = torch.zeros(C_out, H_out, W_out, dtype=torch.int32)

    for oc in range(C_out):
        for h in range(H_out):
            for w in range(W_out):
                acc = 0
                for ic in range(C_in):
                    for kh in range(Kh):
                        for kw in range(Kw):
                            acc += int(x_pad_0[ic, h+kh, w+kw].item()) * \
                                   int(weight_int[oc, ic, kh, kw].item())
                out[oc, h, w] = acc

    return out


# ======================
# 5. 整体测试：读 txt → 卷积 → 对比 output.txt
# ======================

act_path = "activation.txt"
wgt_path = "weight.txt"
out_path = "output.txt"

x_pad_0 = read_activation_txt(act_path, C_in=8)
weight_int = read_weight_txt(wgt_path, C_in=8, K=3)
out_from_txt = read_output_txt(out_path, C_out=8)

print("x_pad_0 shape:", x_pad_0.shape)       # 期望 [8,6,6]
print("weight_int shape:", weight_int.shape) # 期望 [8,8,3,3]
print("out_from_txt shape:", out_from_txt.shape)  # 期望 [8,4,4]

out_manual = manual_conv_3x3_stride1_pad1_from_xpad(x_pad_0, weight_int)

diff = (out_manual.to(torch.int32) - out_from_txt.to(torch.int32)).abs()
print("max abs diff:", diff.max().item())
print("all equal?:", bool(torch.all(diff == 0)))


x_pad_0 shape: torch.Size([8, 6, 6])
weight_int shape: torch.Size([8, 8, 3, 3])
out_from_txt shape: torch.Size([8, 4, 4])
max abs diff: 411
all equal?: False
